# FLISR & Restoration Optimization with Graph Algorithms

From the [Sisyphean Gridworks ML Playground](https://sgridworks.com/ml-playground/guides/05-flisr-restoration.html)

## Setup

Clone the repository and install dependencies. Run this cell first.

In [ ]:
!git clone https://github.com/SGridworks/Dynamic-Network-Model.git 2>/dev/null || echo 'Already cloned'
%cd Dynamic-Network-Model
!pip install -q pandas numpy matplotlib seaborn scikit-learn xgboost lightgbm pyarrow

## Build the Network Graph

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load SP&L datasets using the data loader API
from demo_data.load_demo_data import (
    load_network_nodes, load_network_edges, load_customers, load_outage_history
)

nodes = load_network_nodes()
edges = load_network_edges()
customers = load_customers()
outages = load_outage_history()

# Build a NetworkX graph from the edges table
G = nx.Graph()

for edge_id, row in edges.iterrows():
    G.add_edge(row["from_node_id"], row["to_node_id"],
               edge_id=edge_id, edge_type=row["edge_type"],
               feeder_id=row["feeder_id"])

print(f"Network graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

## Add Switching Devices and Customer Counts

In [ ]:
# Mark nodes that are switching devices
switch_types = ["recloser", "sectionalizer", "tie_switch", "fuse"]
switching_nodes = nodes[nodes["equipment_class"].isin(switch_types)]

# Add switch info to graph nodes
for node_id, row in switching_nodes.iterrows():
    if node_id in G:
        G.nodes[node_id]["has_switch"] = True
        G.nodes[node_id]["switch_type"] = row["equipment_class"]

# Add customer counts per feeder (from customers table)
customer_counts = customers.groupby("feeder_id").size().reset_index(name="customer_count")

# Add node attributes
for node_id, row in nodes.iterrows():
    if node_id in G:
        G.nodes[node_id]["feeder_id"] = row["feeder_id"]
        G.nodes[node_id]["node_type"] = row["node_type"]

print(f"Switching devices found: {len(switching_nodes)}")

## Visualize the Network

In [ ]:
# Use node coordinates from the nodes table
pos = {}
for node_id, row in nodes.iterrows():
    if node_id in G:
        pos[node_id] = (row["longitude"], row["latitude"])

# Focus on one feeder for visualization
feeder_id = "F01"
feeder_nodes = [n for n in G.nodes if G.nodes[n].get("feeder_id") == feeder_id]
feeder_subgraph = G.subgraph(feeder_nodes)
feeder_pos = {n: pos[n] for n in feeder_nodes if n in pos}

# Color nodes: switches in red, others by type
node_colors = ["red" if G.nodes[n].get("has_switch") else "steelblue"
               for n in feeder_subgraph.nodes]

fig, ax = plt.subplots(figsize=(12, 10))
nx.draw(feeder_subgraph, feeder_pos, ax=ax, node_size=20,
        node_color=node_colors, edge_color="#cccccc", width=0.5)
ax.set_title(f"SP&L Feeder {feeder_id} Network")
plt.tight_layout()
plt.show()

## Simulate a Fault

Let's simulate what happens when a line segment fails. We remove the faulted edge from the graph and count how many customers lose power.

In [ ]:
# Find the source node (substation or breaker) for this feeder
source_nodes = nodes[nodes["node_type"].str.contains("substation|breaker", na=False)]
source_node = source_nodes[source_nodes["feeder_id"] == feeder_id].index[0]
print(f"Source node for {feeder_id}: {source_node}")

# Pick a feeder edge to simulate a fault on
feeder_edges = edges[edges["feeder_id"] == feeder_id]
fault_row = feeder_edges.iloc[8]  # pick an edge partway down the feeder
fault_edge = (fault_row["from_node_id"], fault_row["to_node_id"])

print(f"Simulating fault on edge: {fault_edge}")

# Remove the faulted edge
G_faulted = G.copy()
G_faulted.remove_edge(*fault_edge)

# Which nodes can still reach the source?
if source_node in G_faulted and nx.has_path(G_faulted, source_node, fault_edge[0]):
    powered_nodes = nx.node_connected_component(G_faulted, source_node)
else:
    powered_nodes = set()

de_energized = set(feeder_subgraph.nodes()) - powered_nodes

# Count affected customers using the customers table
affected_cust = customers[customers["feeder_id"] == feeder_id]
print(f"\nNodes de-energized: {len(de_energized)}")
print(f"Feeder {feeder_id} total customers: {len(affected_cust):,}")

## Implement FLISR Logic

Now build the automatic FLISR algorithm. The three steps are: (1) Locate the fault, (2) Isolate the faulted section by opening the nearest switches, (3) Restore power to unfaulted sections by closing tie switches.

In [ ]:
def flisr_response(G, fault_edge, source=source_node):
    """Simulate FLISR: isolate fault and restore via alternate path."""

    # Step 1: LOCATE — identify the faulted segment
    print(f"FAULT DETECTED on {fault_edge}")

    # Step 2: ISOLATE — find nearest upstream and downstream switches
    G_work = G.copy()
    G_work.remove_edge(*fault_edge)

    # Find switching device nodes near the fault
    isolation_switches = []
    for n in G.nodes():
        if G.nodes[n].get("has_switch"):
            # Check if this switch is close to the fault
            try:
                path = nx.shortest_path(G, fault_edge[0], n)
                if len(path) 4:
                    isolation_switches.append(
                        (n, G.nodes[n]["switch_type"]))
            except nx.NetworkXNoPath:
                pass

    print(f"ISOLATING via switches: {isolation_switches}")

    # Step 3: RESTORE — find alternate feed paths
    # Check if any tie switch can reconnect de-energized nodes
    components = list(nx.connected_components(G_work))
    source_component = [c for c in components if source in c][0]
    island_components = [c for c in components if source not in c]

    restored_count = 0
    for island in island_components:
        island_size = len(island)
        restored_count += island_size
        print(f"RESTORED island of {island_size} nodes via alternate feed")

    return restored_count

restored = flisr_response(G, fault_edge)
print(f"\nTotal nodes restored by FLISR: {restored}")

## Calculate CMI Savings

In [ ]:
# Use outage history instead of crew dispatch data
# The outage_history already has fault_detected, service_restored, and duration_hours

flisr_time_minutes = 1  # automated switching

# Calculate CMI savings for equipment failure and tree contact outages
sample_outages = outages[outages["cause_code"].isin(
    ["equipment failure", "tree contact"]
)].head(20)

for _, event in sample_outages.iterrows():
    manual_minutes = event["duration_hours"] * 60
    cust = event["affected_customers"]

    cmi_manual = cust * manual_minutes
    cmi_flisr  = cust * flisr_time_minutes
    cmi_saved  = cmi_manual - cmi_flisr

    print(f"Event: {cust:>4} customers, "
          f"Manual: {manual_minutes:>6.0f} min, "
          f"CMI saved: {cmi_saved:>10,.0f}")

## Replay a Storm Event

In [ ]:
# Find a Major Event Day (MED) in the outage data
outages_df = outages.copy()
outages_df["date"] = outages_df["fault_detected"].dt.date
daily_counts = outages_df.groupby("date").size()
storm_day = daily_counts.idxmax()

storm_events = outages_df[outages_df["date"] == storm_day]
print(f"Storm day: {storm_day}")
print(f"Outage events that day: {len(storm_events)}")
print(f"Total customers affected: {storm_events['affected_customers'].sum():,}")

# Calculate total CMI without FLISR vs with FLISR
total_cmi_manual = 0
total_cmi_flisr  = 0

for _, event in storm_events.iterrows():
    duration = event["duration_hours"] * 60
    cust = event["affected_customers"]
    total_cmi_manual += cust * duration
    total_cmi_flisr  += cust * min(duration, flisr_time_minutes)

print(f"\nStorm day CMI (manual):  {total_cmi_manual:>12,.0f}")
print(f"Storm day CMI (FLISR):   {total_cmi_flisr:>12,.0f}")
print(f"CMI reduction:           {total_cmi_manual - total_cmi_flisr:>12,.0f}")
print(f"Improvement:             {((total_cmi_manual - total_cmi_flisr) / total_cmi_manual * 100):.1f}%")

## What You Built and Next Steps